# Notebook des différentes modélisations

Doit être exécuté après celui de Récupération pour pouvoir fonctionner. Il est plus logique de le lire après celui de statistiques descriptives.

In [1]:
import warnings
import requests
import zipfile
import io
import pandas as pd
from shutil import rmtree
from jyquickhelper import add_notebook_menu
from unidecode import unidecode
import numpy as np

import sklearn.metrics
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

add_notebook_menu()

In [2]:
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

## Préambule : Choix sur la base de données

Comme evoqué au début, nous offrons la ici la possibilité de ne garder que les villes de plus de 1000 habitants (aux deux periodes) pour une question de cohérence. En effet, les villes de moins de 1000 habitants peuvent suivre un mode de scrutin différent. Par ailleurs, dans une petite ville, le maire a plus de chance d'être réélu sur des variables non incluses de par sa proximité plus forte avec ses concitoyens.

In [3]:
# Variable booléenne
#Si True, on ne garde que les villes de plus de 1000 habitants
grosse_Ville = True

Dès que cela est possible, nous allons créer une variable valant l'évolution d'évolution entre 2014 et 2019.
Cela va être possible si la variable existe aux deux périodes.

Exemple : Pop_diff = population_2019 - population_2013 (Ces noms de variables sont de circonstances et non les mêmes que dans notre base).

In [4]:
# Prenons-nous en compte l'évolution de 2013 à 2019 ?
# Mettre la variable à True si cela est souhaité
dynamique_temporelle = True

Afin de savoir où en est l'exécution du notebook, il est possible d'activer ce paramètre.

In [5]:
Suivi = True

## 0 - Création de la base d'entraînement et de celle de test

In [6]:
insee = pd.read_csv('Données élections et INSEE.csv')

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


In [7]:
# On crée la dataframe avec les observations où le maire s'est représenté
insee2 = insee[insee["Tentative de réélection"]==True]

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Gestion de la taille des villes considérées :

In [8]:
if grosse_Ville == True :
    
    insee2 = insee2[insee2["P19_POP"] >= 1000]    
    insee2 = insee2[insee2["P13_POP"] >= 1000]

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Nous avons besoin du vecteur contenant la variable à prévoir.

In [9]:
# On crée le vecteur contenant résultat
Reelection = insee2["Nom de l'élu en 2020"] == insee2["Nom de l'élu en 2014"]

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


In [10]:
# On va réduire insee2 en ne gardant que les données de 2014 et 2020 qui sont des valeurs chiffrées
# Valeurs chiffrées
temp = insee2.select_dtypes(include = ["int64", "float64"])
insee2 = pd.concat([insee2["CODGEO"], temp], axis = 1)
# Années gardées
for i in insee2.columns :
    if i[0:3]!="P19" and i[0:3]!="P13" and i!="CODGEO":
        insee2 = insee2.drop(i, axis = 1)
        
# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Si l'on veut être surs que nos modèles soient comparables, il faut centrer et réduire nos données.

In [11]:
temp2 = insee["CODGEO"]

var = insee2.std()
moy = insee2.mean()

insee2 = (insee2-moy)/var
insee2["CODGEO"]=temp2

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


In [12]:
# Certaines de nos colonnes contiennent (énormément) de NaNs.
# Seulement quelques colonnes, redondantes entre 2013 et 2020, sont concernées.
# Par simplicité, nous les enlevons donc.
liste = []
for i in insee2.columns:
        if insee2[i].isnull().values.any()==True :
            liste.append(i)
            
insee2 = insee2.drop(liste, axis = 1)

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Gestion de la prise en compte ou non de la dynamique entre 2013 et 2019 :

In [13]:
if dynamique_temporelle == True :
    
    liste = []
    # On fait la liste des variables qui existent aux deux périodes
    for i in insee2.columns :
        
        if i[0:3] == "P19" :
            temp = "P13" + i[3:]
            
            for j in insee2.columns :
                if j == temp :
                    liste.append(i[3:])
                    break

# On ajoute les variables dynamiques possibles à insee2
for k in liste :
    n1 = "P19" + k
    n2 = "P13" + k
    n3 = k + "_Diff_13_19"
    insee2[n3] = insee2[n1] - insee2[n2]
    
# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


On crée maintenant les bases d'entraînement et de test pour nos différentes modélisations.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(insee2, Reelection, test_size=0.2, random_state=6)

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Nous allons tester plusieurs méthodes. Afin de savoir laquelle est la "meilleure", nous allons les comparer grâce à quatre indicateurs : l'exactitude (accuracy), la précision, le rappel (recall) ainsi que la statistique F1.

L'exactitude regarde à quel point les données simulées sont proches de la réalité et regarde donc la proportion des résultats dont l'issue est la bonne.

La précision correspond au nombre de vrais positifs sur le nombre de prédictions positives.

Le rappel correspond au nombre de vrais positifs sur le nombre réel de positifs.

La statistique F1 correspond à $\frac{2 \times precision \times rappel}{precision + rappel}$.

In [15]:
# Création de la base contenant ces données :
Stats_Modeles = pd.DataFrame(columns = ["Modèle", "Accuracy", "Precision", "Recall", "Score F1", "Coefficients tous nuls ?", "Nombre de coefficients non nuls"])

# Prend un vecteur et renvoie le nombre d'éléments non nuls de ce vecteur
# eps_ou_non est un booléen.
#S'il vaut True, on ne considère pas une églité stricte, mais si la distance entre deux valurs est supérieure à eps ou non
def nombre_Non_Nuls(vect, eps = 0, eps_ou_non = False):
    
    non_nul = 0
    if eps_ou_non == False :
        for i in range(len(vect)):
            if vect[i] != 0 :
                non_nul = non_nul + 1
                
    else :
        for i in range (len(vect)):
            if (abs(vect[i])) > 0 :
                non_nul = non_nul + 1
            
    return non_nul

# Ajoute une ligne à la dataframe avec les statistiques du modèle
# M correspond au nom du modèle
# y_t correspond à l'échantillon connu 
# y_p correspond à l'échantillon prédis
# C dit si les coefficients sont nuls s'ils existent (ce qui correspond à un modèle constat)
# nb_C donne le nombre de coefficients non nuls s'ils existent
def plusUneLigne(M, y_t, y_p, C, nb_C):
    
    # On évalue les stats du modèle
    ac = sklearn.metrics.accuracy_score(y_t, y_p)
    prec = sklearn.metrics.precision_score(y_t, y_p)
    recall = sklearn.metrics.recall_score(y_t, y_p)
    f1 = sklearn.metrics.f1_score(y_t, y_p)
    

    return Stats_Modeles.append({"Modèle" : M,
                                          "Accuracy" : ac,
                                          "Precision" : prec,
                                          "Recall" : recall,
                                          "Score F1" : f1,
                                          "Coefficients tous nuls ?" : C,
                                          "Nombre de coefficients non nuls" : nb_C},
                                          ignore_index=True)

# Calcule le modèle et l'ajoute à la liste recensant l'efficacité des différents modèles
# clf correspond au modèle utilisé
# nom correspond au nom du modèle
# X_te et y_te aux variables de test
# X_tr, y_tr aux variables d'entraînement
# coeffs est un booléen selon de si le modèle utilise des coefficients
# binarisation est un booléen informant de s'il faut binariser le modèle
def modelise_Puis_Resume(clf, nom, X_tr, y_tr, X_te, y_te, coeffs, binarisation):
    
    # Calcul du modèle
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    
    # Binarisation si nécessaire des modèles autres
    if binarisation == True :
        for j in range(len(y_pred)):
            if y_pred[j]<0.5:
                y_pred[j] = 0
            else :
                y_pred[j] = 1
                
    # Prise en compte des informations des coefficients si le modèle en utilise
    nul = np.nan
    non_nuls = np.nan
    if coeffs == True :
        if clf.coef_.ndim == 1:
            nul = np.array_equal(clf.coef_, np.zeros(X_te.shape[1]))
            non_nuls = nombre_Non_Nuls(clf.coef_)
            
        elif clf.coef_.ndim == 2 and clf.coef_.shape[0] == 1 :
            nul = np.array_equal(clf.coef_[0], np.zeros(X_te.shape[1]))
            non_nuls = nombre_Non_Nuls(clf.coef_[0])
        else :
            print("Modèle pas encore compatible")
 
    S_M = plusUneLigne(nom, y_te, y_pred, nul, non_nuls)
    return S_M
    

## 1 - Méthode du lasso

Nous avons ici une base de données avec beaucoup de variables. De ce fait, avec la méthode du lasso, qui ajoute une pénalisation correspondant à la norme 2 du vecteur des $\beta_i$. La formule du lasso est la suivante :
$ \hat{\beta} = argmin_{\beta \in \mathbb{R}^k } = \frac{1}{n} \sum_{i=1}^{n}{(Y_i - X_i^{'}  \beta)^2 + \lambda ||\beta||_1}$ où $\lambda \in \mathbb{R}^{+}$ est un coefficient de pondération. Plus $\lambda$ est grand, plus le nombre de coefficients $\beta_i$ égal à zéro augmente.

Ce modèle est continue donc on ne trouvera pas un $\hat{y}_i \in \{ 0; 1\}$ mais dans $[\ 0; 1]\ $ correspondant plutôt à une probabilité d'être réélu(e). On binarise alors notre vecteur en donnant la valeur de un aux probabilités supérieures à $\frac{1}{2}$ et zéro à celles inférieures : $ \hat{y_i}' = \mathbb{1}_{ [\ \frac{1}{2}; 1]\ }(y_i) $.

In [16]:
lambdas = [ 0.1, 0.001, 0.0001, 0.00001, 0]

for i in lambdas :
    clf = linear_model.Lasso(alpha=i)
    nom_temp = "Lasso (α = " + str(i)+ ")"
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, True, True)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")
    

Modèle  Lasso (α = 0.1)  effectué
Modèle  Lasso (α = 0.001)  effectué
Modèle  Lasso (α = 0.0001)  effectué
Modèle  Lasso (α = 1e-05)  effectué
Modèle  Lasso (α = 0)  effectué


## 2 - Régression logistique

Le second modèle que nous allons utiliser correspond au modèle logistique.

In [17]:
solvers = ["lbfgs", "liblinear", "sag" ]

for i in solvers:
    clf = LogisticRegression(random_state=0, solver = i, max_iter = 1000)
    nom_temp = "Logistique (solver = " + i + ")"
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, True, False)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")

Modèle  Logistique (solver = lbfgs)  effectué
Modèle  Logistique (solver = liblinear)  effectué
Modèle  Logistique (solver = sag)  effectué


## 3 - Méthode des k plus proches voisins

La méthode des k plus proches voisins ($ k \in \mathbb{N}^* $) regarde l'issue la plus présente parmi les k observations qui sont les plus proches de notre observation donc l'issue est à prévoir. La notion de proche peut varier suivant la distance choisie.

In [18]:
combien_voisins = [5, 11, 101, 1001]   

for i in combien_voisins :    
    clf = KNeighborsClassifier(n_neighbors=i)
    nom_temp = str(i) + " plus proches voisins"
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, False, False)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")

Modèle  5 plus proches voisins  effectué
Modèle  11 plus proches voisins  effectué
Modèle  101 plus proches voisins  effectué
Modèle  1001 plus proches voisins  effectué


## 4 - Modèle SVM

On essaie également ce modèle vu en cours qui essaie de séparer les données en divers hyperplans.

In [19]:
kernels = ["linear", "rbf", "sigmoid", "poly"]

for i in kernels :

    clf = SVC(kernel = i, degree = 10)
    nom_temp = "SVM (kernel = " + i + ")"
    
    if i == "linear" :
        coeffic = True
    else :
        coeffic = False
    
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, coeffic, False)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")

Modèle  SVM (kernel = linear)  effectué
Modèle  SVM (kernel = rbf)  effectué
Modèle  SVM (kernel = sigmoid)  effectué
Modèle  SVM (kernel = poly)  effectué


In [20]:
Stats_Modeles

,Modèle,Accuracy,Precision,Recall,Score F1,Coefficients tous nuls ?,Nombre de coefficients non nuls
0,Lasso (α = 0.1),0.841073,0.841073,1.000000,0.913677,True,0
1,Lasso (α = 0.001),0.839009,0.840745,0.997546,0.912458,False,56
2,Lasso (α = 0.0001),0.835913,0.843816,0.987730,0.910119,False,337
3,Lasso (α = 1e-05),0.831785,0.843882,0.981595,0.907544,False,809
4,Lasso (α = 0),0.830753,0.843717,0.980368,0.906924,False,988
5,Logistique (solver = lbfgs),0.826625,0.840168,0.980368,0.904870,False,988
6,Logistique (solver = liblinear),0.826625,0.840168,0.980368,0.904870,False,988
7,Logistique (solver = sag),0.839009,0.840745,0.997546,0.912458,False,988
8,5 plus proches voisins,0.824561,0.840549,0.976687,0.903519,NaN,NaN
9,11 plus proches voisins,0.834881,0.840083,0.992638,0.910011,NaN,NaN


Blabla.

## 5 - Réseau de neurones

Pour les réseaux de neurones, nous allons nous servir de la régression lasso pour présélectionner des variables. Avec le tableau précédent, on obtient que pour α = 0.0001, 337 coefficients sont non-nuls. On va sélectionner ces colonnes pour pouvoir travailler avec.

In [21]:
lambda_337 = 0.0001

clf = linear_model.Lasso(alpha=lambda_337)
nom_temp = "Lasso (α = " + str(lambda_337)+ ")"

Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, True, True)

### Préparation des données

In [22]:
def selection_variable(coef):
    if coef==0:
        return False
    return True

In [23]:
# Préparation des données pour X

coefs = [True] + [selection_variable(coef) for coef in clf.coef_]
# Le premier True sert à selectionner CODGEO, les autres coefficients viennent de la régression lasso
X = insee2.loc[:, coefs]

# Un premier nettoyage des données avait déjà été opéré, mais il est répété à cause des manipulations qui y ont été faites après
X = X.select_dtypes(include=np.number)
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.fillna(0)
col_names = X.columns.values
features = X[col_names]
features = StandardScaler().fit(features.values).transform(features.values)
X[col_names] = features
X

,P13_ACTOCC15P_2ROUES,P13_ACTOCC15P_COMMUN,P13_ACTOCC15P_ILT1,P13_ACTOCC15P_ILT3,P13_ACTOCC15P_ILT4,P13_ACTOCC15P_ILT5,P13_ACTOCC15P_MARCHE,P13_ACTOCC15P_TP,P13_AINACT1564,P13_ANEM_RP_GRAT,...,_RPMAISON_ACH05_Diff_13_19,_RPMAISON_ACH19_Diff_13_19,_RPMAISON_ACH70_Diff_13_19,_RPMAISON_ACH90_Diff_13_19,_RP_1P_Diff_13_19,_RP_4P_Diff_13_19,_RP_GARL_Diff_13_19,_RP_GRAT_Diff_13_19,_RP_LOCHLMV_Diff_13_19,_SCOL1824_Diff_13_19
2,0.106001,0.259940,0.360250,0.527978,-0.038549,-0.105747,0.400678,0.390638,0.428344,0.042586,...,-0.908665,-2.011354,-0.292008,-2.546828,1.615686,-0.512123,-0.076167,-0.731350,1.746927,0.659612
3,-0.183068,-0.184299,-0.185530,0.048791,-0.283289,-0.146479,-0.225680,-0.229515,-0.294402,-0.302832,...,-0.210221,-0.070321,0.048402,-0.165334,0.023840,0.000407,0.276371,0.186707,0.171542,0.094987
8,-0.190943,-0.190884,-0.199360,-0.120087,-0.323827,0.065608,-0.238198,-0.293991,-0.300120,-0.308229,...,0.163356,-0.338995,-0.024352,-0.033161,-0.010238,-0.139256,0.090905,-0.176221,-0.052868,-0.015816
15,-0.172871,-0.183887,-0.182186,0.023002,-0.283869,-0.146519,-0.195615,-0.257195,-0.273563,-0.262755,...,0.139294,-0.228784,0.047392,0.023346,-0.009739,0.111975,0.078748,0.051759,-0.120827,0.137012
16,-0.197567,-0.184786,-0.182601,-0.197918,-0.333230,-0.112648,-0.187511,-0.281497,-0.282474,-0.215467,...,-0.425778,-0.093473,0.383524,-0.267670,-0.008232,0.040090,0.031133,-0.140090,-0.080171,0.191983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27595,0.114508,-0.007839,0.321483,-0.230608,-0.296643,-0.129577,0.458369,0.597449,1.613193,2.658157,...,-1.788685,-0.116246,-3.930223,7.599079,-0.178670,3.496184,1.056987,1.755801,1.674686,0.537333
27596,-0.168255,-0.165242,0.095366,-0.230608,-0.205643,-0.150610,0.289302,0.085008,0.577983,0.809953,...,-0.255253,-0.721367,-1.453661,-0.981500,-0.090450,-0.061284,0.025125,1.267414,0.275510,-0.279510
27597,0.921253,0.051858,2.739565,-0.230608,-0.048120,-0.068640,1.869994,3.477212,7.159557,7.098274,...,8.029524,0.928154,-1.611394,-10.481984,-0.204549,-0.689752,-1.812818,11.818237,1.403548,-3.528268
27598,-0.096989,-0.114219,-0.006236,-0.230608,-0.278086,-0.146437,0.148769,0.128671,0.365701,0.764077,...,-1.141678,0.268807,-0.708157,-0.325769,0.068626,-0.050748,0.423311,0.611606,0.151193,-0.279851


In [24]:
# Préparation des données pour y

y = pd.DataFrame(Reelection)[0].reset_index(drop=True).apply(lambda x: int(x))
y

0       1
1       1
2       1
3       1
4       0
       ..
4838    1
4839    1
4840    1
4841    1
4842    1
Name: 0, Length: 4843, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Modélisation

In [26]:
def Construction_modele(X_train, y_train, hparam1=256, hparam2='relu'):
    #Construction couches de neurones
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(hparam1, activation=hparam2))
    model.add(tf.keras.layers.Dense(hparam1, activation=hparam2))
    #Softmax permet d'obtenir une probabilité
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    #Ajustement modèle et choix de métrique
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    
    #Entraînement du modèle
    model.fit(X_train, y_train, epochs=50, verbose=0)
    return model

In [27]:
# On cherche à obtenir la meilleure valeur possible pour les hyperparamètres

def fine_tune_coef(X_train, y_train, X_test, y_test, hparam1=256, hparam2='relu'):
    réseau = Construction_modele(X_train, y_train, hparam1, hparam2)
    test_loss, test_acc = réseau.evaluate(X_test,  y_test, verbose=2)
    return test_acc, réseau

def fine_tune(X_train, y_train, X_test, y_test, param_dict):
    acc_max, model_max = fine_tune_coef(X_train, y_train, X_test,  y_test, hparam1 = 256, hparam2 = 'relu')    
    for i in range(3):
        for j in range(3):
            hp1 = list(param_dist.values())[0][i]
            hp2 = list(param_dist.values())[1][j]
            test_acc, réseau = fine_tune_coef(X_train, y_train, X_test,  y_test, hparam1=hp1, hparam2=hp2)

            if test_acc > acc_max:
                acc_max = test_acc
                model_max = réseau
                
    return réseau

In [28]:
# "A la main", on cherche à optimiser la valeur des hyperparamètres pour obtenir la meilleure précision (accuracy)

# Dictionnaire des paramètres à essayer
hparams1 = [128, 256, 512]
hparams2 = ['elu', 'relu', 'sigmoid']
epochs = [50,60,70]
param_dist = dict(hparams1=hparams1, 
                  hparams2=hparams2,
                  epochs=epochs)

réseau = fine_tune(X_train, y_train, X_test, y_test, param_dist)

2022-12-20 23:39:54.285584: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


38/38 - 0s - loss: 0.4812 - accuracy: 0.8357 - 91ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4584 - accuracy: 0.8390 - 73ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4823 - accuracy: 0.8423 - 71ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4330 - accuracy: 0.8439 - 75ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4664 - accuracy: 0.8332 - 72ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4723 - accuracy: 0.8415 - 69ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4322 - accuracy: 0.8439 - 70ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4860 - accuracy: 0.8448 - 88ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4972 - accuracy: 0.8365 - 89ms/epoch - 2ms/step
38/38 - 0s - loss: 0.4318 - accuracy: 0.8439 - 75ms/epoch - 2ms/step


### Résultats

In [29]:
test_loss, test_acc = réseau.evaluate(X_test,  y_test, verbose=2)

print('\nMeilleure accuracy:', round(test_acc*100, 2), '%')
print('Loss associée:', round(test_loss*100, 2), '%')

38/38 - 0s - loss: 0.4318 - accuracy: 0.8439 - 32ms/epoch - 849us/step

Meilleure accuracy: 84.39 %
Loss associée: 43.18 %


In [30]:
print('Selon les données disponibles,', round(sum(Reelection)/insee2.shape[0]*100, 2),'% des maires des villes de plus de 1000 habitants candidats à leurs propres successions se sont fait réélire en 2020.')

Selon les données disponibles, 85.44 % des maires des villes de plus de 1000 habitants candidats à leurs propres successions se sont fait réélire en 2020.


La prédiction (triviale) qui consiste à dire que chaque maire se représentant sera réélu, tombera juste 85% du temps, ce qui est équivalent aux résultats obtenus avec ce réseau de neurones. Appliquer un réseau de neurones dans ce cadre n'apporte donc aucun pouvoir de prédiction supplémentaire. En effet, conformément à ce qui a été montré avec les statistiques descriptives, les données de l'INSEE dont nous disposons, mêmes locales, ne sont que très faiblement corrélées aux performances des maires. Cela explique la faiblesse de l'approche par réseau de neurones ici.

## 6 - Analyse

### + pistes quelles données pourraient mieux fonctionner ?